<a href="https://colab.research.google.com/github/Matrix7043/Machine_learning101/blob/main/Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import typing
import pandas as pd
import numpy as np
from numpy import ndarray

In [2]:
def sigmoid(X: ndarray) -> ndarray:
    return 1/(1 + np.exp(-1.0 * X))


In [58]:
def zscore_normalize_features(X: ndarray) -> ndarray:

    mu     = np.mean(X, axis=0)
    sigma  = np.std(X, axis=0)
    X_norm = (X - mu) / sigma

    return (X_norm, mu, sigma)

In [ ]:
df0 = pd.read_csv('/content/sample_data/california_housing_train.csv')
df1 = df0.drop(columns=['median_house_value'])
df2 = df0['median_house_value']
x = df1.to_numpy()
y = df2.to_numpy().reshape(len(df1), 1)
xnorm, xmu, xsig = zscore_normalize_features(x)
ynorm, ymu, ysig = zscore_normalize_features(y)
print(df0.corr(method='pearson'))

In [4]:
def forward_propagation(X_batch: ndarray,
                        Y_batch: ndarray,
                        weights: dict[str, ndarray]) -> tuple[dict[str, ndarray], float]:
    #W1 = np.transpose(weights['W1'])
    #W2 = np.transpose(weights['W2'])

    M1 = np.dot(X_batch, weights['W1'])
    # print("M1", M1.shape)

    N1 = M1 + weights['B1']
    # print("N1", N1.shape)

    O1 = sigmoid(N1)
    # print("O1", O1.shape)

    M2 = np.dot(O1, weights['W2'])
    # print("M2", M2.shape)

    P = M2 + weights['B2']
    # print("P", P.shape)

    L = np.mean(np.power((Y_batch - P), 2))

    forward_info: dict[str, ndarray] = {}
    forward_info['X'] = X_batch
    forward_info['Y'] = Y_batch
    forward_info['W1'] = weights['W1']
    forward_info['W2'] = weights['W2']
    forward_info['M1'] = M1
    forward_info['M2'] = M2
    forward_info['N1'] = N1
    forward_info['O1'] = O1
    forward_info['P'] = P

    return forward_info, L


In [5]:
def back_propogation(forward_info: dict[str, ndarray],
                     weights: dict[str, ndarray]) -> dict[str, ndarray]:

    dLdP = -1 * (forward_info['Y'] - forward_info['P'])
    # print("Y", forward_info['Y'])
    # print("P", forward_info['P'])
    # print("dLdP", dLdP)

    dPdM2 = np.ones_like(forward_info['M2'])
    # print("dPdM2", dPdM2.shape)

    dM2dO1 = np.transpose(forward_info['W2'])
    # print("dM2dO1", dM2dO1.shape)

    dO1dN1 = sigmoid(forward_info['N1'] * (1 - sigmoid(forward_info['N1'])))
    # print("dO1dN1", dO1dN1.shape)

    dN1dM1 = np.ones_like(forward_info['M1'])
    # print("dN1dM1", dN1dM1.shape)

    dM1dW1 = np.transpose(forward_info['X'])
    # print("dM1dW1", dM1dW1.shape)

    dN1dB1 = np.ones_like(weights['B1'])
    # print("dN1dB1", dN1dB1.shape)

    dM2dW2 = np.transpose(forward_info['O1'])
    # print("dM2dW1", dM2dW2)

    dPdB2 = np.ones_like(weights['B2'])
    # print("dPdB2", dPdB2.shape)

    dLdM2 = dLdP * dPdM2
    # print("dLdM2", dLdM2.shape)

    dLdO1 = np.dot(dLdM2, dM2dO1)
    # print("dLdO1", dLdO1.shape)

    dLdM1 = (dLdO1 * dO1dN1) * dN1dM1
    # print("dLdM1", dLdM1.shape)

    # Important

    dLdW1 = np.dot(dM1dW1, dLdM1)

    dLdW2 = np.dot(dM2dW2, dLdP)

    dLdB1 = ((dLdO1 * dO1dN1) * dN1dB1).sum(axis=0)

    dLdB2 = (dLdP * dPdB2).sum(axis=0)

    loss_gradient: dict[str, ndarray] = {}
    loss_gradient['W1'] = dLdW1
    loss_gradient['W2'] = dLdW2
    loss_gradient['B1'] = dLdB1
    loss_gradient['B2'] = dLdB2

    return loss_gradient


In [7]:
def predict(X: ndarray,
            weights: dict[str, ndarray]) -> ndarray:

    M1 = np.dot(X, weights['W1'])
    N1 = M1 + weights['B1']
    O1 = sigmoid(N1)
    M2 = np.dot(O1, weights['W2'])
    P = M2 + weights['B2']

    return P

In [8]:
def test(X_test: ndarray,
         Y_test: ndarray,
         weights: dict[str, ndarray]) -> tuple[float, float]:

    def mae(Y: ndarray,
            P: ndarray) -> float:

        return np.mean(Y - P)

    def rmse(Y: ndarray,
             P: ndarray) -> float:

        return np.power(np.mean(np.power(P - Y, 2)), 1/2)

    P = predict(X_test, weights)

    return (rmse(Y_test, P), mae(Y_test, P))

In [ ]:
def train(X_batch: ndarray,
          Y_batch: ndarray,
          learning_rate: float = 0.00000001,
          hidden_size: int = 13,
          iterations: int = 5000,
          split: float = 0.7) -> dict[str, ndarray]:

    total = X_batch.shape[0]
    num = int(total*split)

    X_train: ndarray = X_batch[:num:]
    Y_train: ndarray = Y_batch[:num:]
    # print(X_train.shape)
    # print(Y_train.shape)

    X_test: ndarray = X_batch[num::]
    Y_test: ndarray = Y_batch[num::]
    # print(X_test.shape)
    # print(Y_test.shape)

    def init_weights(input_size: int,
                     hidden_size: int) -> dict[str, ndarray]:

              weights: dict[str, ndarray] = {}
              weights['W1'] = np.random.randn(input_size, hidden_size)
              weights['B1'] = np.random.randn(1, hidden_size)
              weights['W2'] = np.random.randn(hidden_size, 1)
              weights['B2'] = np.random.randn(1)

              #print("W1", weights['W1'].shape)
              #print("B1", weights['B1'].shape)
              #print("W2", weights['W2'].shape)
              #print("B2", weights['B2'].shape)
              return weights

    weights = init_weights(X_train.shape[1], hidden_size)

    for i in range(iterations):

        forward_info, loss = forward_propagation(X_train, Y_train, weights)
        loss_grads = back_propogation(forward_info, weights)

        if i%10 == 0:
            print(loss)

        for key in weights.keys():
            weights[key] -= learning_rate * loss_grads[key]

    theta = test(X_test, Y_test, weights)
    print(theta)


In [66]:
train(xnorm, ynorm)

(11900, 8)
(11900, 1)
(5100, 8)
(5100, 1)
2.0307490732557203
2.0191743402198172
2.007719307782981
1.9963826044347255
1.9851628749538983
1.9740587802083591
1.963068996957258
1.9521922176558835
1.9414271502630414
1.9307725180509319
1.9202270594174864
1.909789527701128
1.8994586909979283
1.8892333319811183
1.8791122477229312
1.8690942495187286
1.8591781627133954
1.84936282652996
1.8396470939004144
1.8300298312986996
1.8205099185758291
1.8110862487971235
1.8017577280815205
1.7925232754429343
1.7833818226336386
1.7743323139896443
1.7653737062780395
1.7565049685462752
1.7477250819733576
1.7390330397229306
1.7304278467982175
1.7219085198988033
1.7134740872792182
1.705123588609318
1.6968560748364163
1.688670608049161
1.6805662613431227
1.6725421186880765
1.6645972747969477
1.6567308349964078
1.6489419150990907
1.6412296412774148
1.6335931499389817
1.626031587603538
1.6185441107814695
1.6111298858538208
1.6037880889538076
1.5965179058498087
1.589318531829812
1.5821891715873069
1.575129039108587